<a href="https://colab.research.google.com/github/CamelGoong/NLP/blob/main/Chapter2_Preparing_NLP_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#01. Tensorflow

더미 데이터를 활용한 감정 분석 모델링(Sentiment Analysis)

>Method1. Sequential API

In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']

labels = [[1], [0], [1], [1], [0], [1]]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples) # 위에서 tokenizer한 samples들을 sequences로 변환

word_index = tokenizer.word_index

In [ ]:
# 모델 구축에 필요한 변수 정의
batch_size = 2
num_epochs = 100
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

In [ ]:
import tensorflow.keras.layers as layers
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length = 4))
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)))
model.add(layers.Dense(hidden_dimension, activation = 'relu'))
model.add(layers.Dense(output_dimension, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 1s 6ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6740 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6562 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6372 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6179 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5942 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5686 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5361 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 0.5001 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.4625 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

>Method2. Functional API

In [ ]:
inputs = layers. Input(shape = (4,)) # 먼저 Input 모듈을 정의해야함.
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis = 1)
hidden_layer = layers.Dense(hidden_dimension, activation = 'relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation = 'sigmoid')(hidden_layer)
model = tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 1s 4ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6760 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6616 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6495 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6319 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6137 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5929 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 0.5692 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5385 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 11ms/step - loss: 0.5039 - accuracy: 1.0000
Epoch 11/100
3/3 [==========

>Method3. Custom Model

In [ ]:
class CustomModel(tf.keras.Model):
  def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
    super(CustomModel, self).__init__(name = 'my_model')
    self.embedding = layers.Embedding(vocab_size, embed_dimension)
    self.dense_layer = layers.Dense(hidden_dimension, activation = 'relu')
    self.output_layer = layers.Dense(output_dimension, activation = 'sigmoid')

  def call(self, inputs):
    x = self.embedding(inputs)
    x = tf.reduce_mean(x, axis = 1)
    x = self.dense_layer(x)
    x = self.output_layer(x)
    return x

model = CustomModel(vocab_size = vocab_size, embed_dimension = emb_size, hidden_dimension = hidden_dimension, output_dimension = output_dimension)
model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 1s 5ms/step - loss: 0.6952 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6745 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6596 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6441 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6263 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6053 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5810 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 0.5528 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5211 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.4839 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

#02. Scikit-learn

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris_dataset = load_iris()
print("iris_dataset key: {}".format(iris_dataset.keys()))

iris_dataset key: dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [3]:
print("shape of data: {}".format(iris_dataset['data'].shape))

shape of data: (150, 4)


In [4]:
print(iris_dataset['feature_names'])

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [5]:
print(iris_dataset['target_names'])
print(iris_dataset['target'])

['setosa' 'versicolor' 'virginica']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [6]:
print(iris_dataset['DESCR'])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [7]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_label, test_label = train_test_split(iris_dataset['data'], iris_dataset['target'], test_size = 0.25, random_state = 7)
print("shape of train_input: {}".format(train_input.shape))
print("shape of test_input: {}".format(test_input.shape))
print("shape of train_label: {}".format(train_label.shape))
print("shape of test_label: {}".format(test_label.shape))

shape of train_input: (112, 4)
shape of test_input: (38, 4)
shape of train_label: (112,)
shape of test_label: (38,)


>지도학습(KNN classifier)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(train_input, train_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [10]:
import numpy as np
# 임의의 샘플 1개(with feature값 4개) 생성
new_input = np.array([[6.1, 2.8, 4.7, 1.2]])

In [11]:
knn.predict(new_input)

array([1])

In [12]:
# 전에 분리해둔 평가데이터로 성능 측정
predict_label = knn.predict(test_input)

In [17]:
print("Test Accuracy: {:.2f}%".format(np.mean(predict_label == test_label)*100))

Test Accuracy: 92.11%


>비지도학습(K-means)

In [18]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters = 3) # 애초에 붓꽃 데이터의 분류 자체가 0, 1, 2의 3개이므로, 3으로 설정.
k_means.fit(train_input)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [20]:
# 이런 식으로 분류된 데이터의 라벨을 확인가능
k_means.labels_

array([0, 2, 2, 1, 2, 2, 1, 0, 1, 1, 2, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0, 0,
       0, 1, 2, 1, 1, 0, 2, 0, 0, 2, 1, 0, 2, 0, 1, 2, 1, 0, 1, 0, 2, 2,
       1, 0, 0, 1, 2, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 0, 2, 1, 1, 2, 1, 0,
       0, 1, 0, 0, 1, 2, 2, 1, 1, 0, 2, 0, 0, 1, 1, 2, 0, 1, 1, 1, 2, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1, 1, 2,
       1, 0], dtype=int32)

In [21]:
print("0 Cluster: ", train_label[k_means.labels_ == 0])
print("1 Cluster: ", train_label[k_means.labels_ == 1])
print("2 Cluster: ", train_label[k_means.labels_ == 2])

0 Cluster:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
1 Cluster:  [1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 2 1 2 1 1 1 1 1 1 1 2 1 2 1 1 1 1
 2 1 1 1 2 1 1]
2 Cluster:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [22]:
import numpy as np
# 임의의 샘플데이터 1개 생성
new_input = np.array([[6.1, 2.8, 4.7, 1.2]])

In [23]:
k_means.predict(new_input)

array([1], dtype=int32)

In [24]:
predict_cluster = k_means.predict(test_input)

In [26]:
print("Test Accuracy: {:.2f}%".format(np.mean(predict_cluster == test_label)*100))

Test Accuracy: 78.95%


>특징 추출

>>CountVectorizer

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

text_data = ['나는 배가 고프다', '내일 점심 뭐먹지', ' 내일 공부 해야겠다', '점심 먹고 공부 해야지']

countvectorizer = CountVectorizer()

In [28]:
countvectorizer.fit(text_data)
print(countvectorizer.vocabulary_)

{'나는': 2, '배가': 6, '고프다': 0, '내일': 3, '점심': 7, '뭐먹지': 5, '공부': 1, '해야겠다': 8, '먹고': 4, '해야지': 9}


In [30]:
sentence = [text_data[0]]
print(countvectorizer.transform(sentence).toarray())

[[1 0 1 0 0 0 1 0 0 0]]


>>TfidfVectorizer

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

In [33]:
tfidf.fit(text_data)
print(tfidf.vocabulary_)

{'나는': 2, '배가': 6, '고프다': 0, '내일': 3, '점심': 7, '뭐먹지': 5, '공부': 1, '해야겠다': 8, '먹고': 4, '해야지': 9}


In [35]:
sentence = [text_data[1]]
print(tfidf.transform(sentence).toarray())

[[0.         0.         0.         0.52640543 0.         0.66767854
  0.         0.52640543 0.         0.        ]]


#03. NLP Tokenizing tools

>English Tokenizing Library(NLTK, Spacy)



>>1) NLTK

In [40]:
import nltk
nltk.download("book", quiet=True)
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [42]:
# 단어 단위 토크나이징
from nltk.tokenize import word_tokenize
sentence = "Natural language processing (NLP) is a subfield of computer science, information engineering, and artificial inteliigence cocerned with the interactionsbetween computer and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data."

print(word_tokenize(sentence))

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', ',', 'information', 'engineering', ',', 'and', 'artificial', 'inteliigence', 'cocerned', 'with', 'the', 'interactionsbetween', 'computer', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data', '.']


In [45]:
from nltk.tokenize import sent_tokenize
paragraph = sentence

print(sent_tokenize(paragraph))

['Natural language processing (NLP) is a subfield of computer science, information engineering, and artificial inteliigence cocerned with the interactionsbetween computer and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.']


>>2) Spacy

In [47]:
import spacy


In [48]:
nlp = spacy.load('en')

doc = nlp(sentence)

In [49]:
word_tokenized_sentence = [token.text for token in doc]
sentence_tokenized_sentence = [sent.text for sent in doc.sents]
print(word_tokenized_sentence)
print(sentence_tokenized_sentence)

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', ',', 'information', 'engineering', ',', 'and', 'artificial', 'inteliigence', 'cocerned', 'with', 'the', 'interactionsbetween', 'computer', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data', '.']
['Natural language processing (NLP) is a subfield of computer science, information engineering, and artificial inteliigence cocerned with the interactionsbetween computer and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.']


Korean Tokenizing Library(KoNLPy)

In [51]:
!pip install konlpy
import konlpy
from konlpy.tag import Okt

okt = Okt()

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 56.2 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [53]:
text = "한글 자연어 처리는 재밌다 이제부터 열심히 해야지ㅎㅎㅎ"
print(okt.morphs(text))
print(okt.morphs(text, stem = True))
print(okt.nouns(text))
print(okt.phrases(text))

['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '해야지', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '하다', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '이제']
['한글', '한글 자연어', '한글 자연어 처리', '이제', '자연어', '처리']


In [54]:
print(okt.pos(text))
print(okt.pos(text, join = True))

[('한글', 'Noun'), ('자연어', 'Noun'), ('처리', 'Noun'), ('는', 'Josa'), ('재밌다', 'Adjective'), ('이제', 'Noun'), ('부터', 'Josa'), ('열심히', 'Adverb'), ('해야지', 'Verb'), ('ㅎㅎㅎ', 'KoreanParticle')]
['한글/Noun', '자연어/Noun', '처리/Noun', '는/Josa', '재밌다/Adjective', '이제/Noun', '부터/Josa', '열심히/Adverb', '해야지/Verb', 'ㅎㅎㅎ/KoreanParticle']


In [55]:
# konlpy 데이터
from konlpy.corpus import kolaw
from konlpy.corpus import kobill

kolaw.open('constitution.txt').read()[:20]

'대한민국헌법\n\n유구한 역사와 전통에 '

In [56]:
kobill.open('1809891.txt').read()

'국가공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9891\n\n발의연월일 : 2010.  11.  12.  \n\n발  의  자 : 정의화․이명수․김을동 \n\n이사철․여상규․안규백\n\n황영철․박영아․김정훈\n\n김학송 의원(10인)\n\n제안이유 및 주요내용\n\n  초등학교 저학년의 경우에도 부모의 따뜻한 사랑과 보살핌이 필요\n\n한 나이이나, 현재 공무원이 자녀를 양육하기 위하여 육아휴직을 할 \n\n수 있는 자녀의 나이는 만 6세 이하로 되어 있어 초등학교 저학년인 \n\n자녀를 돌보기 위해서는 해당 부모님은 일자리를 그만 두어야 하고 \n\n이는 곧 출산의욕을 저하시키는 문제로 이어질 수 있을 것임.\n\n  따라서 육아휴직이 가능한 자녀의 연령을 만 8세 이하로 개정하려\n\n는 것임(안 제71조제2항제4호).\n\n- 1 -\n\n\x0c법률  제        호\n\n국가공무원법 일부개정법률안\n\n국가공무원법 일부를 다음과 같이 개정한다.\n\n제71조제2항제4호 중 “만 6세 이하의 초등학교 취학 전 자녀를”을 “만 \n\n8세 이하(취학 중인 경우에는 초등학교 2학년 이하를 말한다)의 자녀를”\n\n로 한다.\n\n부      칙\n\n이 법은 공포한 날부터 시행한다.\n\n- 3 -\n\n\x0c신 ·구조문대비표\n\n현      행\n\n개   정   안\n\n제71조(휴직) ① (생  략)\n\n제71조(휴직) ① (현행과 같음)\n\n  ② 임용권자는 공무원이 다음 \n  ② -------------------------\n\n각 호의 어느 하나에 해당하는 \n\n----------------------------\n\n사유로 휴직을 원하면 휴직을 \n\n----------------------------\n\n명할 수 있다. 다만, 제4호의 \n\n-------------.---------------\n\n경우에는 대통령령으로 정하는 \n\n--------------------------

#4. 그 밖의 라이브러리(전처리)